## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 128
num_epoch  = 180

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0001

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [60, 120, 150]

pretrained_model       = './cifar100_resnet56_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## (Resnet56) model

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarResnetConfig(num_layers = 56, cifar = 100)
model = model_gen.GetCifarResnet()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 4.005834, Test Acc: 11.74 %
2 epoch end, loss: 3.378891, Test Acc: 21.28 %
3 epoch end, loss: 2.912839, Test Acc: 25.34 %
4 epoch end, loss: 2.555351, Test Acc: 34.30 %
5 epoch end, loss: 2.314694, Test Acc: 34.63 %
6 epoch end, loss: 2.134416, Test Acc: 40.12 %
7 epoch end, loss: 1.977506, Test Acc: 42.94 %
8 epoch end, loss: 1.871201, Test Acc: 44.84 %
9 epoch end, loss: 1.758121, Test Acc: 46.41 %
10 epoch end, loss: 1.674446, Test Acc: 49.89 %
11 epoch end, loss: 1.593361, Test Acc: 49.22 %
12 epoch end, loss: 1.527852, Test Acc: 50.64 %
13 epoch end, loss: 1.477492, Test Acc: 51.38 %
14 epoch end, loss: 1.425636, Test Acc: 50.88 %
15 epoch end, loss: 1.381236, Test Acc: 50.42 %
16 epoch end, loss: 1.332052, Test Acc: 55.32 %
17 epoch end, loss: 1.300440, Test Acc: 55.13 %
18 epoch end, loss: 1.262632, Test Acc: 55.69 %
19 epoch end, loss: 1.233343, Test Acc: 55.86 %
20 epoch end, loss: 1.198897, Test Acc: 55.00 %
21 epoch end, loss: 1.175391, Test Acc: 57.44 %
2

171 epoch end, loss: 0.030247, Test Acc: 69.58 %
172 epoch end, loss: 0.029395, Test Acc: 69.45 %
173 epoch end, loss: 0.029645, Test Acc: 69.61 %
174 epoch end, loss: 0.029980, Test Acc: 69.52 %
175 epoch end, loss: 0.030294, Test Acc: 69.60 %
176 epoch end, loss: 0.028160, Test Acc: 69.35 %
177 epoch end, loss: 0.029118, Test Acc: 69.61 %
178 epoch end, loss: 0.028991, Test Acc: 69.44 %
179 epoch end, loss: 0.029360, Test Acc: 69.31 %
180 epoch end, loss: 0.028616, Test Acc: 69.44 %

Training is finished!


In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 69.44 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)